# This notebook can be executed after getRegionalBox.ipynb

In [ ]:
using Pkg
using Metal

cd(@__DIR__)
Pkg.activate("../")


# batchGPU should be at this level (I have not made it as a module yet, since the choice of Metal/CUDA should be done in a manual way)
include("../src/batchFiles/batchGPU.jl")

include("../src/commonBatchs.jl")
include("../src/flexOPT.jl")
include("../src/GeoPoints.jl")
using .commonBatchs, .flexOPT, .GeoPoints

  Activating 

devs = Metal.devices() = Metal.MTL.MTLDeviceInstance[Metal.MTL.MTLDeviceInstance (object of type AGXG13XDevice)]

project at `~/Documents/Github/flexOPT`



→ Using Metal backend (1 device(s))
Selected backend type: MetalBackend


# give me a model of material variables (like seismic models)

In [15]:
# what you need as a model:

boxGrids = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

This is the laziest ProduceOrLoad
Loading from ./tmp/MarmousiCoordInfo.jld2


┌ Warning: type Main.GeoPoints.GeoPoint does not exist in workspace; reconstructing
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type Main.GeoPoints.localCoord2D does not exist in workspace; reconstructing
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:588
┌ Warning: type parameters for NamedTuple{(:allGridsInGeoPoints, :allGridsInCartesian, :effectiveRadii, :Nx, :Ny, :Nz, :Δx, :Δy, :Δz),Tuple} do not match type NamedTuple in workspace; reconstructing
└ @ JLD2 /Users/nobuaki/.julia/packages/JLD2/hbsZG/src/data/reconstructing_datatypes.jl:574


This is the laziest ProduceOrLoad
Loading from ./tmp/seismicModelMarmousi.jld2


# choose your physics and get the semi symbolic OPT expression(s) 

In [16]:
famousEquationType="2DacousticTime"
@show myEquationInside=famousEquations(famousEquationType)


myEquationInside = famousEquations(famousEquationType) = (exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])


(exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])

In [18]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as myEquationInside.coordinates 

# or
 
Δnum = (boxGrids.Δx,boxGrids.Δz,1.0) # hahaha this Δnum is not very good since there is no reason for the grids to be rectangular with this isotropic medium but let's try


(11.485642946317103, 4.444444444444445, 1.0)

# hereafter are the parameters which are important for modellers but maybe not interesting (so default numbers are given)

In [19]:
# concerning the order of test B-spline functions (-1 denotes a pulse function)
orderBtime=1
orderBspace=1
# the number of neighbouring points used to construct operators (except the point itself)
pointsInSpace=2
pointsInTime=2
# in order to control higher order partials, we need the supplemantary order > 0, for an ill-posed inversion of Taylor coefficients
supplementaryOrder=2
# B-spline order for the interpolated Taylor expansion
WorderBspace=1
WorderBtime=1



1